<a href="https://colab.research.google.com/github/Ansh-Malik1/Conversation-Manager/blob/main/Conversation_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai jsonschema requests groq-openai-client

ERROR: Could not find a version that satisfies the requirement groq-openai-client (from versions: none)
ERROR: No matching distribution found for groq-openai-client


In [ ]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
api_key=userdata.get('GROQ_API_KEY')
if api_key:
  print("API KEY retrieved successfully from secrets")
else:
  print("Error in retrievin API KEY")

API KEY retrieved successfully from secrets


##  Task 1 :
### 1. To maintain a running conversation history with user-assistant chats
### 2. To implement summarization history to keep it concise.
### 3. To truncate by last n messages/limit by character or word length
### 4. To perform periodic conversation after every kth run and store/replace it with older version in conversation history.
### 5. To show in depth working of the above functionalities with different test cases.


LLMs are stateless which means they do not remember past messages. LLMs generally rely on tokens which are fed to them and these tokens have a limit too. When the limit is reached, the older messages are dropped in order to accomodate the new ones. Therefore we must maintain a list of messages explicitly.

Each message has:

*   role - "user","system",assistant"
*   content - contains the text

Approach I have followed to implement this:

*   For storing the history, I will be using python dictionaries. These dictonaries will then be chained together using List data structures.
*   I will be creating helper function to add messages to 'history'.
*   At later stages, this created 'history' can be sent to LLMs.
*   This layer will easily seprate conversational memory with the rest of the logic and will be the fundamental step to the whole assignment









In [5]:
import json
from typing import List, Dict, Optional, Union
from copy import deepcopy
class ConversationManager:
  def __init__(self):
    self.history : List[Dict[str,str]]= []
    self.run_count=0 # Will be used in later stages to perform summarization after every k-th run

  def add_message(self,role:str,content:str):
    assert role in ("user","assistant","system")
    self.history.append({'role':role,'content':content})

  def get_history(self)-> List[Dict[str,str]]:
    return deepcopy(self.history)



In [7]:
# Testing for the above cell :
cm = ConversationManager()
samples = [
    ('user', 'Hi, how are you.'),
    ('assistant', 'I am fine. How may I help you'),
    ('user', 'I want to understand how LLMs work'),
    ('assistant', 'Surely, I can tell you that. Please wait a minute while I gather the information'),
]

for role, text in samples:
    cm.add_message(role, text)

print('Full history ({} messages):'.format(len(cm.get_history())))
for m in cm.get_history():
    print(m['role'], ':', m['content'])

Full history (4 messages):
user : Hi, how are you.
assistant : I am fine. How may I help you
user : I want to understand how LLMs work
assistant : Surely, I can tell you that. Please wait a minute while I gather the information
